**<font size="4">Here is a glimpse of the approach I followed in the M-5 Forecasting Accuracy Competition to achieve a Leaderboard position of 2% (silver medal) in my first Kaggle competition:</font>**

<font size="3">I started the competition just a week before the final submission deadline, so I didn't had the time to write everything from scratch, which I was going to do before starting. Instead, I looked for information others have already shared, followed the discussions and used the knowledge gained to create my base model. Then I added my own features to that base model, and with some parameter tuning, it turned out to be a great one. Through this approach, I actually gained much more knowledge and applied learning than I could get working fully from scratch.     
    To those who want a short answer, the key to achieve good results is feature extraction. To those intrested in actual workflow, please follow along. Go through M5-Forecasting Accuracy competition from Walmart before that https://www.kaggle.com/c/m5-forecasting-accuracy/overview/description. Since I used information from many notebooks and discussions, it wouldn't be feasible to reference everyone, but I am thankful to the community, and therefore, am adding to it.

The steps I followed are shown in subsequent sections:</font>

In [ ]:
import os
import pandas as pd
import numpy as np
import plotly_express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import warnings
warnings.filterwarnings('ignore')
from lightgbm import LGBMRegressor
import joblib
from sklearn.metrics import mean_squared_error

<font size="4">1. Extract the data</font>

In [ ]:
sales = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv')
sales.name = 'sales'
calendar = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/calendar.csv')
calendar.name = 'calendar'
prices = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sell_prices.csv')
prices.name = 'prices'

<font size="3">Add new prediction rows, i.e. 1942-1969, which is what we are predicting</font>

In [ ]:
for d in range(1942,1970):
    col = 'd_' + str(d)
    sales[col] = 0
    sales[col] = sales[col].astype(np.int16)

<font size="4">2. Save the kernel from memory overflow risk</font>

<font size="3">If you use the default datatypes assigned to the kernels by dataframe, you will most likely run into memory allocation error, considering such huge dataset. This is what I think is a must for people with less computing power in their PC. I also changed datatypes of all my added features based on convenience/precision required such that minimum possible memory is allocated.</font>

In [ ]:
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == np.object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df  

sales = downcast(sales)
prices = downcast(prices)
calendar = downcast(calendar)

<font size="4">3. Convert the data from wide to long format (Melting)</font>

<font size="3">The conversion to long format enables easier transformation operations to columns, and we can easily monitor the columns for their feature importances/ other factors due to their accessability. </font>

In [ ]:
df = pd.melt(sales, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='sold').dropna()

Combine the three datasets:

In [ ]:
df = pd.merge(df, calendar, on='d', how='left')
df = pd.merge(df, prices, on=['store_id','item_id','wm_yr_wk'], how='left')

Before moving on towards the best part, i.e. feature extraction, let's store the category codes of the category columns we added:

In [ ]:
d_id = dict(zip(df.id.cat.codes, df.id))
d_item_id = dict(zip(df.item_id.cat.codes, df.item_id))
d_dept_id = dict(zip(df.dept_id.cat.codes, df.dept_id))
d_cat_id = dict(zip(df.cat_id.cat.codes, df.cat_id))
d_store_id = dict(zip(df.store_id.cat.codes, df.store_id))
d_state_id = dict(zip(df.state_id.cat.codes, df.state_id))

<font size="4">4. Feature Extraction</font>

<font size="3">This is the part I liked the most, the part that I mostly did on my own, and the part that helped me achieve the tier. Adding new meaningful features or removing others based on their respective feature importances is the approach I used for this competition.

The following features are added/converted/deleted:</font>


<font size="3">a) Transform events with a label encoder and convert to category datatype, use only the number present in the day feature as an integer to get an insight on the time dependence, convert categorical features to respective codes and use day-of-month from the dates feature as others are already there.</font>

In [ ]:
list1=['event_name_1','event_type_1','event_name_2','event_type_2']
from sklearn.preprocessing import LabelEncoder
for i in list1:
    df[i] = df[i].cat.add_categories("nan").fillna("nan")
    df[i]=LabelEncoder().fit_transform(df[i]).astype(np.int8)


In [ ]:
df.d = df['d'].apply(lambda x: x.split('_')[1]).astype(np.int16)

In [ ]:
cols = df.dtypes.index.tolist()
types = df.dtypes.values.tolist()
for i,type in enumerate(types):
    if type.name == 'category':
        df[cols[i]] = df[cols[i]].cat.codes

In [ ]:
# extracting day_of_week has shown some memory errors, therefore I have dropped the date, 
# it is a good feature though, try to incorporate it and let me know the reason for error 
# df['date'] = df['date'].apply(lambda x: x.strftime('%d')).astype(np.int8)
df.drop(['date'],axis=1, inplace=True)

In [ ]:
#make datatype of event as category
for i in list1:
    df[i]=df[i].astype('category')

<font size="3">b) Introduce lags for number of items sold </font>

Lags help us in a time series dataset by enabling to use previous values of a feature as aother feature.           
**Note**: Use only **lags starting from 28 days** as any less than that is going to give poor test results since some of the 28 prediction columns will remain 0.

In [ ]:
# I have used 4 lags here in intervals of 7, all showed a good value of feature importance
lags = [28,35,42,49]
for lag in lags:
    df['sold_lag_'+str(lag)] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],as_index=False)['sold'].shift(lag).astype(np.float16)

<font size="3">c) Introduce lags for events</font>

My reasoning for this is that people usually do shopping a day before, or weekend before an important event and not on the event itself. Also the sales may dip immediately after the day of event (it was just a guess but gave a good feature to use)

In [ ]:
# I have added 4 days surrounding the event as features
lags2 = [-2,-1,1,2]
for lag in lags2:
    df['event1_lag_'+str(lag)] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],as_index=False)['event_name_1'].shift(lag).astype(np.float16)
    df['event1_lag_'+str(lag)].fillna(100, inplace=True)
    df['event1_lag_'+str(lag)]=df['event1_lag_'+str(lag)].astype(np.int8)
    df['event1_lag_'+str(lag)]=df['event1_lag_'+str(lag)].astype('category')
# event type didn't showed a good feature importance, opposite to event itself
# for lag in lags2:
#     df['eventtype1_lag_'+str(lag)] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],as_index=False)['event_type_1'].shift(lag).astype(np.float16).fillna(100, inplace=True)
#     df['eventtype1_lag_'+str(lag)].fillna(100, inplace=True)
#     df['eventtype1_lag_'+str(lag)]=df['eventtype1_lag_'+str(lag)].astype(np.int8)
#     df['eventtype1_lag_'+str(lag)]=df['eventtype1_lag_'+str(lag)].astype('category')

<font size="3">d) Apply mean encodings, atleast of the 12 aggregations they have taken in their WRMSE document</font>

In [ ]:
df['item_sold_avg'] = df.groupby('item_id')['sold'].transform('mean').astype(np.float16)    
df['state_sold_avg'] = df.groupby('state_id')['sold'].transform('mean').astype(np.float16)    #total 3 unique values, 1 for each state
df['store_sold_avg'] = df.groupby('store_id')['sold'].transform('mean').astype(np.float16)  #10 unique values
df['cat_sold_avg'] = df.groupby('cat_id')['sold'].transform('mean').astype(np.float16)
df['dept_sold_avg'] = df.groupby('dept_id')['sold'].transform('mean').astype(np.float16)
df['cat_dept_sold_avg'] = df.groupby(['cat_id','dept_id'])['sold'].transform('mean').astype(np.float16)
df['store_item_sold_avg'] = df.groupby(['store_id','item_id'])['sold'].transform('mean').astype(np.float16)
df['cat_item_sold_avg'] = df.groupby(['cat_id','item_id'])['sold'].transform('mean').astype(np.float16)
df['dept_item_sold_avg'] = df.groupby(['dept_id','item_id'])['sold'].transform('mean').astype(np.float16)
df['state_store_sold_avg'] = df.groupby(['state_id','store_id'])['sold'].transform('mean').astype(np.float16)
df['state_store_cat_sold_avg'] = df.groupby(['state_id','store_id','cat_id'])['sold'].transform('mean').astype(np.float16)
df['store_cat_dept_sold_avg'] = df.groupby(['store_id','cat_id','dept_id'])['sold'].transform('mean').astype(np.float16)

<font size="3">e) Currently week id is not encoded and has large values, taking higher space. Reduce the space by encoding with integers (int64->int8).</font>

In [ ]:
df['wm_yr_wk_linear']=LabelEncoder().fit_transform(df['wm_yr_wk'].values).astype(np.int16)

df.drop(['wm_yr_wk'], axis=1, inplace=True)

<font size="3">f) Add a price-diff feature which gives the increase/decrease in price of an item at a store from it's previous week value. Its reasoning is pretty obvious.</font>

In [ ]:
df['price_lag'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],as_index=False)['sell_price'].shift(7).astype(np.float16)
df['price-diff']=df['price_lag']-df['sell_price']
df.drop(['price_lag'], axis=1, inplace=True)

<font size="3">g) Decimal feature which shows the decimal part of the price. I just used it to gain more insights from price feature as I think that's the most important feature here (it may show some user interaction with prices, discrepancies with change, etc)</font>

In [ ]:
df['sell_price'].fillna(-1,inplace=True)
df['decimal']=df['sell_price'].apply(lambda x: 100*(x-int(x))).astype(np.int16)
df['sell_price'].replace(-1,np.nan,inplace=True)

<font size="3">h) Difference of price of an item at a store from it's current trend/tragectory </font>

In [ ]:
df['expanding_price_mean'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sell_price'].transform(lambda x: x.expanding(2).mean()).astype(np.float16)
df['diff_moving_mean']=df['expanding_price_mean']-df['sell_price']
df.drop(['expanding_price_mean'], axis=1, inplace=True)

Some more memory reduction to save the kernel:

In [ ]:
df['price-diff']=df['price-diff'].astype(np.float16)
df.drop(['wday'], axis=1, inplace=True)
df['decimal']=df['decimal'].astype(np.int8)
df['year']=LabelEncoder().fit_transform(df['year']).astype(np.int8)

<font size="3">h) A trend feature for price which gives difference from overall mean, instead of a expending mean as used previously.</font>

In [ ]:
df['daily_avg_sold'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id','d'])['sell_price'].transform('mean').astype(np.float16)
df['avg_sold'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sell_price'].transform('mean').astype(np.float16)
df['selling_trend'] = (df['daily_avg_sold'] - df['avg_sold']).astype(np.float16)
df.drop(['daily_avg_sold','avg_sold'],axis=1,inplace=True)

Drop features with about zero feature importance to save computation

In [ ]:
list3=['cat_id','state_id']
for i in list3:
    df.drop([i], axis=1, inplace=True)

Save Dataframe to output. Remove some start days as they are not as much important and lags there are empty.

In [ ]:
df = df[df['d']>=49]


In [ ]:

df.to_pickle('data.pkl')
del df, sales, prices, calendar
gc.collect()

<font size="4">5. Model preperation and evaluation</font>

In [ ]:
data = pd.read_pickle('data.pkl')
valid = data[(data['d']>=1599) & (data['d']<1942)][['id','d','sold']]  
valid_csv=data[(data['d']>=1914) & (data['d']<1942)][['id','d','sold']]
test = data[data['d']>=1942][['id','d','sold']]
eval_preds = test['sold']
valid_preds = valid['sold']
valid_preds_csv=valid_csv['sold']

Store categorical columns to manually provide as input to LGBM Regressor

In [ ]:
cat_column=[]
for i in data.columns:
    if(str(data.dtypes[i])=='category'):
        cat_column.append(i)

<font size="3">LGBM model training.      
Note: I have tried train test split of 1 year to do some tunning of parameters and then trained the model on all the problem set, i.e. day-1 to day-1914 to recieve output. 
The best approach here will be to do a time series CV, and you may try it if you can compute that.</font>

In [ ]:
for store in d_store_id:
    df = data[data['store_id']==store]
    
    #Split the data
    X_train, y_train = df[df['d']<1914].drop('sold',axis=1), df[df['d']<1914]['sold']
    # Uncover if you want to use last year for train
    #X_valid, y_valid = df[(df['d']>=1599) & (df['d']<1942)].drop('sold',axis=1), df[(df['d']>=1599) & (df['d']<1942)]['sold']
    X_valid_csv, y_valid_csv = df[(df['d']>=1914) & (df['d']<1942)].drop('sold',axis=1), df[(df['d']>=1914) & (df['d']<1942)]['sold']
    X_test = df[df['d']>=1942].drop('sold',axis=1)
    
    #Train and validate
    model = LGBMRegressor(
        learning_rate= 0.05,
        subsample=0.6,
        feature_fraction=0.6,
        num_iterations = 1200,
        max_bin=350,
        num_leaves= 100,
        lambda_l2=0.003,
        max_depth=200,
        min_data_in_leaf= 80,
        force_row_wise= True,
    )
    print('*****Prediction for Store: {}*****'.format(d_store_id[store]))
    model.fit(X_train, y_train, eval_set=[(X_train,y_train),(X_valid_csv,y_valid_csv)],
             eval_metric='rmse',  verbose=100, early_stopping_rounds=20,categorical_feature=cat_column)
    valid_preds_csv[X_valid_csv.index] = model.predict(X_valid_csv)
    eval_preds[X_test.index] = model.predict(X_test)
    filename = 'model'+str(d_store_id[store])+'.pkl'
    # save model
    joblib.dump(model, filename)
    del model, X_train, y_train, X_valid_csv, y_valid_csv
    gc.collect()

In [ ]:
feature_importance_df = pd.DataFrame()
features = [f for f in data.columns if f != 'sold']
for filename in os.listdir('/kaggle/working/'):
    if 'model' in filename:
        # load model
        model = joblib.load(filename)
        store_importance_df = pd.DataFrame()
        store_importance_df["feature"] = features
        store_importance_df["importance"] = model.feature_importances_
        store_importance_df["store"] = filename[5:9]
        feature_importance_df = pd.concat([feature_importance_df, store_importance_df], axis=0)
    
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (averaged over store predictions)')
    plt.tight_layout()
    
display_importances(feature_importance_df)

<font size="4">6. Submission</font>

In [ ]:
#Get the actual validation results
valid_csv['sold'] = valid_preds_csv
validation = valid_csv[['id','d','sold']]
validation = pd.pivot(validation, index='id', columns='d', values='sold').reset_index()
validation.columns=['id'] + ['F' + str(i + 1) for i in range(28)]
validation.id = validation.id.map(d_id).str.replace('evaluation','validation')

#Get the evaluation results
test['sold'] = eval_preds
evaluation = test[['id','d','sold']]
evaluation = pd.pivot(evaluation, index='id', columns='d', values='sold').reset_index()
evaluation.columns=['id'] + ['F' + str(i + 1) for i in range(28)]
#Remap the category id to their respective categories
evaluation.id = evaluation.id.map(d_id)

#Prepare the submission
submit = pd.concat([validation,evaluation]).reset_index(drop=True)
# submit.memory_usage().sum()
submit.to_csv('submission.csv',index=False)

Do some post processing in the sub file also, like replacing the predictions below zero with zero as they are just increasing your error. Had I seen this before, I'd have been on the gold tier instead of silver. Analyzing the excel/result file is important.

<font size="3">Thank you all for being patient. I wrote this notebook in relation with the M-5 competition but I wanted to provide people, mostly newcomers a general approach to have a good start in Kaggle. The notebooks/discussions are very valuable for general practical learning in Machine Learning and I would recommend every newcomer/member to involve in them to gain more knowledge.</font>